In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from custom_py_modules import *


In [10]:
# Load three years of data

# 5 years in dataframe
df_load = pd.concat([pd.read_csv('./test_simulink/E-load-2013.csv'), 
                     pd.read_csv('./test_simulink/E-load-2014.csv'), 
                     pd.read_csv('./test_simulink/E-load-2015.csv'),
                     pd.read_csv('./test_simulink/E-load-2017.csv'),
                     pd.read_csv('./test_simulink/E-load-2018.csv')
                     ])
df_weather = pd.concat([pd.read_csv('./test_simulink/E-2013.csv'), 
                        pd.read_csv('./test_simulink/E-2014.csv'), 
                        pd.read_csv('./test_simulink/E-2015.csv'),
                        pd.read_csv('./test_simulink/E-2017.csv'),
                        pd.read_csv('./test_simulink/E-2018.csv')
                        ])

# Combine irradiance
df_weather['irradiance_total'] = df_weather['irradiance_direct'] + df_weather['irradiance_diffuse']

# Change the time column names
df_weather['timestamp'] = df_weather['local_time_no_dst']
df_weather = df_weather.drop(columns=['local_time_no_dst'])

df_load['timestamp'] = df_load['DD/MM/YYYY HH:MM']
df_load = df_load.drop(columns=['DD/MM/YYYY HH:MM'])

# Convert to datetime
df_load['timestamp'] = pd.to_datetime(df_load['timestamp'], format='%d/%m/%Y %H:%M')
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], format='%Y-%m-%d %H:%M:%S')
# Set timestamp as index
df_load = df_load.set_index('timestamp')
df_weather = df_weather.set_index('timestamp')

# Add kW column
df_load['Load (kW)'] = df_load['Load (Watt)'] / 1000
df_load_hourly = df_load.resample('H').mean()

df_load_hourly_2013 = df_load_hourly.loc['2013-01-01':'2013-12-31']
df_load_hourly_2014 = df_load_hourly.loc['2014-01-01':'2014-12-31']
df_load_hourly_2015 = df_load_hourly.loc['2015-01-01':'2015-12-31']
df_load_hourly_2017 = df_load_hourly.loc['2017-01-01':'2017-12-31']
df_load_hourly_2018 = df_load_hourly.loc['2018-01-01':'2018-12-31']

df_weather_hourly_2013 = df_weather.loc['2013-01-01':'2013-12-31']
df_weather_hourly_2014 = df_weather.loc['2014-01-01':'2014-12-31']
df_weather_hourly_2015 = df_weather.loc['2015-01-01':'2015-12-31']
df_weather_hourly_2017 = df_weather.loc['2017-01-01':'2017-12-31']
df_weather_hourly_2018 = df_weather.loc['2018-01-01':'2018-12-31']

In [11]:
# Define wind turbines
gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                              performance_coefficient=0.4, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=20, # [kW] 
                              nominal_wind_speed=13, # [m/s]
                              cut_out_speed=20, # [m/s] 
                              cut_in_speed=4, # [m/s] 
                              price=4000, # EUR from alibaba
                              lifetime=22.5, # years 
                              height=18) # [m]

aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                              performance_coefficient=0.35, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=10, # [kW] 
                              nominal_wind_speed=11, # [m/s]
                              cut_out_speed=32, # [m/s] 
                              cut_in_speed=2.5, # [m/s] 
                              price=2500, # EUR 
                              lifetime=25, # years 
                              height=18) # [m]

travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                              performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=1.6, # [kW]
                                nominal_wind_speed=10, # [m/s]
                                cut_out_speed=60, # [m/s]
                                cut_in_speed=2.5 , # [m/s]
                                price=500, # EUR
                                lifetime=25, # years
                                height=12) # [m]

wf_builder = WindFarmBuilder()
example_mixed_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [0, 5, 20])

In [12]:
# In this cell, define the energy system

# Wind farm
sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [10, 0, 0])

# Solar farm
sample_solar_farm = SolarFarm(area=800, # [m^2] 
                              pricepersqm= 100 # in E
                              )

# MicroGrid
sample_microgrid = MicroGrid(windfarm = sample_wind_farm, 
                             solarfarm= sample_solar_farm,
                             storage_capacity_kwh= 350, # [kWh]
                             storage_efficiency=0.8, # [-] 
                             storage_price_per_kw= 2000 # [EUR]
                             )

In [13]:
sample_net_power = sample_microgrid.get_net_power(
    df_hourlyload_1year=df_load_hourly_2013,
    df_weather_hourly_1year=df_weather_hourly_2013,
    plot=False
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Net Power (kW)'], name='Net Power'))
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Net Power Storage (kW)'], name='Net Power with Storage', line=dict(width=2, dash='dash')))
# Add load line
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Load (kW)'], name='Load', line=dict(color='green', width=1)))
# Add power generation lines
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Power Output (kW)'], name='Power Output', line=dict(color='orange', width=1)))
fig.update_layout(title='Net Power Output Over Time',
                  xaxis_title='Timestamp',
                  yaxis_title='Power Output (kW)')
fig.show()

In [14]:
def get_micro_grid_cost_function(solar_price_psqm, 
                                 storage_price_pkw, 
                                 generator_price_per_rated_kw, 
                                 diesel_price, # this kinda serves as penalty for unreliability
                                 gen_fuel_consumption, 
                                 hi_power_turbine_price, 
                                 mid_power_turbine_price, 
                                 low_power_turbine_price, 
                                 list_of_load_dfs,
                                 list_of_weather_dfs,
                                 wf_builder: WindFarmBuilder,
                                 operation_time
                                 ):

    # Define wind 
  gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                              performance_coefficient=0.4, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=20, # [kW] 
                              nominal_wind_speed=13, # [m/s]
                              cut_out_speed=20, # [m/s] 
                              cut_in_speed=4, # [m/s] 
                              price=hi_power_turbine_price, # EUR from alibaba
                              lifetime=22.5, # years 
                              height=18) # [m]

  aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                              performance_coefficient=0.35, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=10, # [kW] 
                              nominal_wind_speed=11, # [m/s]
                              cut_out_speed=32, # [m/s] 
                              cut_in_speed=2.5, # [m/s] 
                              price=mid_power_turbine_price, # EUR 
                              lifetime=25, # years 
                              height=18) # [m]

  travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                              performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=1.6, # [kW]
                                nominal_wind_speed=10, # [m/s]
                                cut_out_speed=60, # [m/s]
                                cut_in_speed=2.5 , # [m/s]
                                price=low_power_turbine_price, # EUR
                                lifetime=25, # years
                                height=12) # [m]

  def cost_function(x):
      """
      x[0] is number of hi power turbines
      x[1] is number of mid power turbines
      x[2] is number of low power turbines
      x[3] is area of solar panels sqm
      x[4] is storage capacity in kWh
      """

      num_hi_power_turbines = int(round(x[0]))
      num_mid_power_turbines = int(round(x[1]))
      num_low_power_turbines = int(round(x[2]))
      # Build MicroGrid
      sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [num_hi_power_turbines, num_mid_power_turbines, num_low_power_turbines])
      sample_solar_farm = SolarFarm(area=x[3], # [m^2] 
                                    pricepersqm= solar_price_psqm # in E
                                    )
      sample_microgrid = MicroGrid(windfarm = sample_wind_farm,
                                    solarfarm= sample_solar_farm,
                                    storage_capacity_kwh= x[4], # [kWh]
                                    storage_efficiency=0.8, # [-] 
                                    storage_price_per_kw= storage_price_pkw # [EUR]
                                    )
      diesel_gen = DieselGenerator(price_per_rated_kw=generator_price_per_rated_kw, # [EUR]
                                  diesel_price=diesel_price, # [EUR]
                                  fuel_consumption=gen_fuel_consumption # [L/kWh]
                                  )
      generator_required_powers = []
      generator_running_costs = []

      for load_df, weather_df in zip(list_of_load_dfs, list_of_weather_dfs):
          net_power = sample_microgrid.get_net_power(df_hourlyload_1year=load_df,
                                                    df_weather_hourly_1year=weather_df,
                                                    plot=False)
          generator_required_powers.append(diesel_gen.get_generator_specs(net_power))

          generator_running_costs.append(diesel_gen.get_running_cost(net_power))
      
      # Calculate total cost = microgrid cost + install cost of diesel generator + fuel cost of diesel generator
      microgrid_cost = sample_microgrid.get_total_price()
      diesel_gen_install_cost = np.max(generator_required_powers) * generator_price_per_rated_kw
      mean_fuel_cost = np.mean(generator_running_costs)
      total_cost = microgrid_cost + diesel_gen_install_cost + mean_fuel_cost * operation_time
      
      return total_cost
  return cost_function

In [16]:
my_cost_function = get_micro_grid_cost_function(solar_price_psqm=100,
                                                storage_price_pkw=2500,
                                                generator_price_per_rated_kw=400,
                                                diesel_price=1.9,
                                                gen_fuel_consumption=0.4,
                                                hi_power_turbine_price=4000,
                                                mid_power_turbine_price=2500,
                                                low_power_turbine_price=500,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

# Get cost of installing the microgrid and cost of running the microgrid for 25 years
total_cost = my_cost_function([10, 0, 0, 1000, 350])
print("Total cost of installing the microgrid and running it for 25 years is: ", total_cost, " EUR")

# Set bounds for the variables
# x[0] is number of hi power turbines
# x[1] is number of mid power turbines
# x[2] is number of low power turbines
# x[3] is area of solar panels sqm
# x[4] is storage capacity in kWh

from scipy.optimize import differential_evolution

# Define bounds for the variables
bounds = [(0, 10),  # Number of high power turbines
          (0, 10),  # Number of mid power turbines
          (0, 10),  # Number of low power turbines
          (0, 20000),  # Area of solar panels in sqm
          (0, 5000)]  # Storage capacity in kWh

# Run the optimization using Differential Evolution
result = differential_evolution(my_cost_function, bounds, maxiter=10, disp=True)

print("Optimization result: ", result)

Total cost of installing the microgrid and running it for 25 years is:  1201545.9779493276  EUR


KeyboardInterrupt: 